In [1]:
# jupyter notebook --NotebookApp.kernel_spec_manager_class='environment_kernels.EnvironmentKernelSpecManager'
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

# Imports
import numpy as np
import tensorflow as tf

from tensorflow.contrib import learn
from tensorflow.contrib.learn.python.learn.estimators import model_fn as model_fn_lib

tf.logging.set_verbosity(tf.logging.INFO)

# Our application logic will be added here

import math
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

In [2]:
# data generating script
def place_cube(matrix, origin, r):
	#dim = matrix.shape()
	matrix[
	origin[0]-r:origin[0]+r,
	origin[1]-r:origin[1]+r,
	origin[2]-r:origin[2]+r,
	] = 1
	return matrix

def dist(x, y):
	return math.sqrt(
		(x[0]-y[0])**2+
		(x[1]-y[1])**2+
		(x[2]-y[2])**2
		)

def place_sphere(matrix, origin, r):
	for index, x in np.ndenumerate(matrix):
	    if dist(index, origin) <= r:
	    	matrix[index] = 1
	return matrix

def random_shapes(m_size,num):
    shapes = np.zeros((m_size,m_size,m_size,num))
    shape_types = np.random.randint(low = 0,high = 2,size= num)
    
    for i in range(num):
        r = np.random.randint(low = 3,high = np.floor(m_size/2-1))
        org = np.random.randint(low = r + 1,high = m_size - r - 1, size = 3)
        if shape_types[i] == 0:
            shapes[:,:,:,i] = place_cube(shapes[:,:,:,i],org,r)
        elif shape_types[i] == 1:
            shapes[:,:,:,i] = place_sphere(shapes[:,:,:,i],org,r)
            
    return shapes, shape_types
        

def matching_shapes(m_size, num):
	cubes = np.zeros((m_size,m_size,m_size,num))
	spheres = np.zeros((m_size,m_size,m_size,num))
	for i in range(num):
		r = np.random.randint(low = 3,high = np.floor(m_size/2-1))
		org = np.random.randint(low = r + 1,high = m_size - r - 1, size = 3)
		cubes[:,:,:,i] = place_cube(cubes[:,:,:,i],org,r)
		spheres[:,:,:,i] = place_sphere(spheres[:,:,:,i],org,r)
	return cubes, spheres

In [3]:
def cnn_model_fn(features, labels, mode):
  #"""Model function for CNN."""
  # Input Layer
    input_layer = tf.reshape(features, [-1, 28, 28, 1])

  # Convolutional Layer #1
    conv1 = tf.layers.conv2d(
      inputs=input_layer,
      filters=32,
      kernel_size=[5, 5],
      padding="same",
      activation=tf.nn.relu)
        

  # Pooling Layer #1
    pool1 = tf.layers.max_pooling2d(inputs=conv1, pool_size=[2, 2], strides=2)

  # Convolutional Layer #2 and Pooling Layer #2
    conv2 = tf.layers.conv2d(
      inputs=pool1,
      filters=64,
      kernel_size=[5, 5],
      padding="same",
      activation=tf.nn.relu)
    pool2 = tf.layers.max_pooling2d(inputs=conv2, pool_size=[2, 2], strides=2)

  # Dense Layer
    pool2_flat = tf.reshape(pool2, [-1, 7 * 7 * 64])
    dense = tf.layers.dense(inputs=pool2_flat, units=1024, activation=tf.nn.relu)
    dropout = tf.layers.dropout(
      inputs=dense, rate=0.4, training=mode == learn.ModeKeys.TRAIN)

  # Logits Layer
    logits = tf.layers.dense(inputs=dropout, units=10)

    loss = None
    train_op = None

  # Calculate Loss (for both TRAIN and EVAL modes)
    if mode != learn.ModeKeys.INFER:
        onehot_labels = tf.one_hot(indices=tf.cast(labels, tf.int32), depth=10)
        loss = tf.losses.softmax_cross_entropy(
            onehot_labels=onehot_labels, logits=logits)

  # Configure the Training Op (for TRAIN mode)
    if mode == learn.ModeKeys.TRAIN:
        train_op = tf.contrib.layers.optimize_loss(
            loss=loss,
            global_step=tf.contrib.framework.get_global_step(),
            learning_rate=0.001,
            optimizer="SGD")

  # Generate Predictions
    predictions = {
        "classes": tf.argmax(
          input=logits, axis=1),
        "probabilities": tf.nn.softmax(
          logits, name="softmax_tensor")
    }

  # Return a ModelFnOps object
    return model_fn_lib.ModelFnOps(
        mode=mode, predictions=predictions, loss=loss, train_op=train_op)
